# Example usage

To use `pyraingen` in a project:

```python
import pyraingen

print(pyraingen.__version__)
```

## Get Example Data

Use the built-in function `pyraingen.get_example_data()` to copy the datasets used for the below examples to the directory of your choice. The defeault is to copy the data to the current working directory. Do not change any of the file names.

```python
from pyraingen.get_example_data import get_example_data

get_example_data(daily=True, subdaily=True, ifd=True)
```

## Generate Daily Rainfall

NB: The `regionaliseddailysim` backend is written in fortran and must be compiled prior to using. A precompiled version for Python 3.8.5 on Windows 64 has been provided with the package, however, if you are using macOS or Linux you will have to use f2py to compile a version suitable for your operating system. If you are on windows it is highly recommened that you create an environment using python 3.8.5 and use the precompiled version. 
If you wish to use another version of python it will require you to compile a version suited to that python version. It is recommened that you use MSYS2 to download MinGW and then download the MinGW toolchain `mingw-w64-x86_64-toolchain` which contains the needed C and fortran compiler. Remeber to add the folder (e.g. `C:\msys64\mingw64\bin`) to your user environment variables path.

Use the inbuilt function `pyraingen.get_for_path()` to get the path to the fortran script and change, or point, to that directory to compile. Use the following commands in the cmd prompt.

`python -m numpy.f2py -c regionalised_daily.for -m regionalised_daily --fcompiler=gnu95 --compiler=mingw32`


Point to the location(s) where you have stored the example data. Please copy the `stn_record.dat` from the example daily data to the working directory where the `regionaliseddailysim` function will be executed from.

```python
from pyraingen.regionaliseddailysim import regionaliseddailysim

nyears = 79
startyear = 1930
nsim = 10
targetidx = 66037 
targetlat = -33.9410 
targetlon = 151.1730 
targetelev = 6.00 
targetdcoast = 0.27 
targetanrf = 1086.71
targettemp = 22.40
data_path = "path/to/daily_data/"

regionaliseddailysim(
    nyears, startyear, nsim,
    targetidx, targetlat, targetlon, 
    targetelev, targetdcoast, targetanrf,
    targettemp, data_path, 
    output_path_nc='daily.nc',
    output_stats="stat_.out", output_val="rev_.out",
    cutoff=0.30, wind=15, nstation=5, 
    nlon=3, lag=1, iamt=1, ival=0, irf=1, rm=1,
    getstations=True
)
```

When using `regionaliseddailysim` for applications other than the example use case, you will need to first find the statistically similar "nearby" stations and get the necessary daily data in the same format as the example data. When running `regionaliseddailysim` the first time through it will display the calculated nearby stations, or you can use the inbuilt function `station()`.

```python
from pyraingen.getnearbystations import station

param = {}
param['nNearStns']       = 5
param['pathStnData']     = None  #Use data provided with package
param['pathModelCoeffs'] = None  #Use data provided with package
param['pathDailyData']   = "path/to/daily_data/"

target = {}
target['index']           = 66037      
target['lat']             = -33.9410      
target['lon']             = 151.1730    
target['elevation']       = 6.00     
target['distToCoast']     = 0.27      
target['annualRainDepth'] = 1086.71      
target['temp']            = 22.40    

station(param, target, nAttributes=33,
    fout=f'nearby_station_details.out'
)
```


An example of the selected nearby stations used for daily generation from Dykman et al (2024) is displayed below. For further explanation of the selection of nearby stations please refer to Dykman et al (2024).

![Nearby Subdaily Stations](nearby_stations_daily.png)

## Perform Subdaily Disaggregation

Point to the location(s) where you have stored the example data.

```python
from pyraingen.regionalisedsubdailysim import regionalisedsubdailysim

fnameInput = "daily.nc"
pathSubDaily = "path/to/sub_daily_data/"
targetIndex = 66037
targetlat = -33.9410 
targetlon = 151.1730 
targetelev = 6.00 
targetdcoast = 0.27 
targetanrf = 1086.71
targettemp = 22.40
fnameSubDaily = "subdaily.nc"

regionalisedsubdailysim(
    fnameInput, pathSubDaily, targetIndex, 
    pathIndex=None, pathCoeff=None, pathReference=' ', 
    fnameSubDaily=fnameSubDaily,
    minYears=10, nYearsPool=500, dryWetCutoff=0.30,
    halfWinLen=15, maxNearNeighb=10, nSims=10,
    genSeqOption=3, nYearsRef=50,
    absDiffTol=0.1, gso3_lat=targetlat, gso3_lon=targetlon,
    gso3_elev=targetelev, gso3_distcoast=targetdcoast, 
    gso3_anrf=targetanrf, gso3_temp=targettemp
)
```

When using `regionalisedsubdailysim` for applications other than the example use case, you will need to first find the statistically similar "nearby" stations and get the necessary subdaily daily data in the same format as the example data. Set ```pathIndex``` and ```pathCoeff``` to NONE to use the paths to data included as part of the `pyraingen` package that is necessary for it to run. Then use the `nearbystations` function to get a list of the set of subdaily station ids for all the seasonal nearest like stations.

```python
from pyraingen.nearbystations import nearbystations

pathIndex = None
pathCoeff = None
targetIndex = 66037

subdailystns = nearbystations(pathIndex, pathCoeff, targetIndex, 
                                lat=targetlat, lon=targetlon, elev=targetelev, 
                                distcoast=targetdcoast, anrf=targetanrf, temp=targettemp) 
```

An example of the selected nearby stations used for subdaily dissaggregation for each season from Dykman et al (2024) is displayed below. For further explanation of the selection of nearby stations please refer to Dykman et al (2024).

![Nearby Subdaily Stations](nearby_stations_subdaily2.png)

## Constrain IFDs

`TargetIFD` data should be a csv file with size (rows = frequencies, cols = durations). No headers or row labels. Return periods should get smaller down the columns and durations should increase from left to right across the rows. E.g.

|  AEP (%) / duration (min) | 30 | 60 | 360 | 720 |
| --- | --- | --- | --- | --- |
| **2**  |     |    |     |    |
| **5**  |    |    |     |    |
| **10**  |    |    |     |    |
| **20**  |    |    |     |    |
| **50**  |    |    |     |    |
| **63.20** |    |    |     |    |

`TargetIFDdurationsEst` are all the durations present in the TargetIFD file. `TargetIFDdurations` are the durations you wish to condition. 

Point to the location(s) where you have stored the example data.

```python
from pyraingen.ifdcond import ifdcond

fileNameInput ="subdaily.nc"
fileNameOutput = "condsubdaily.nc"
fileNameTargetIFD = "targetifds.csv"
TargetIFDdurationsEst = [30, 60, 360, 720]
TargetIFDdurations = [30, 60, 360, 720]
AEP = [63.20, 50, 20, 10, 5, 2]

ifdcond(fileNameInput, fileNameOutput, fileNameTargetIFD, 
        nSims=10, TargetIFDdurationsEst=TargetIFDdurationsEst,
       TargetIFDdurations=TargetIFDdurations, AEP=AEP, plot=True)
```

An example of the output plots comparing raw and conditioned IFD data from Dykman et al (2024) are displayed below.

![ifd cond plot](ifdcond_plot2.png)